In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd
import os
from scipy.signal import butter, filtfilt, iirnotch, welch

# =============================================================================
# 1️⃣ Helper Functions
# =============================================================================

def load_eeg(base_path, subject_id, session_id):
    path = os.path.join(base_path, f"SBJ{subject_id:02d}", f"S{session_id:02d}", "Train")
    try:
        mat = sio.loadmat(os.path.join(path, "trainData.mat"))
        eeg = mat["trainData"]  # shape: (channels, samples, events)
        labels = np.loadtxt(os.path.join(path, "trainTargets.txt"), dtype=int)
        return eeg, labels
    except Exception:
        return None, None


def preprocess_eeg(eeg, fs=250):
    # Band-pass filter (0.5–30 Hz)
    b_band, a_band = butter(4, [0.5, 30], btype="band", fs=fs)
    eeg = filtfilt(b_band, a_band, eeg, axis=1)

    # 50 Hz notch filter
    b_notch, a_notch = iirnotch(50, 30, fs)
    eeg = filtfilt(b_notch, a_notch, eeg, axis=1)

    # Baseline correction (first 200 ms)
    baseline = np.mean(eeg[:, :int(0.2 * fs), :], axis=1, keepdims=True)
    eeg = eeg - baseline
    return eeg


def compute_bandpower(eeg, fs=250):
    # Compute average power in standard EEG frequency bands
    freqs, psd = welch(eeg, fs=fs, axis=1, nperseg=fs*2)
    bands = {"delta": (0.5, 4), "theta": (4, 8), "alpha": (8, 13), "beta": (13, 30)}
    bp = {}
    for band, (low, high) in bands.items():
        idx = np.logical_and(freqs >= low, freqs <= high)
        bp[band] = np.mean(psd[:, idx, :], axis=(1, 2))
    return bp


def compute_p300_features(eeg, fs=250):
    # Compute ERP features: P300 amplitude and latency
    erp = np.mean(eeg, axis=2)
    time = np.arange(eeg.shape[1]) / fs * 1000 - 200  # in ms

    window_mask = (time >= 250) & (time <= 600)
    p300_window = erp[:, window_mask]

    p300_amp = np.max(p300_window, axis=1)
    p300_latency = time[window_mask][np.argmax(p300_window, axis=1)]
    return p300_amp, p300_latency


# =============================================================================
# 2️⃣ Iterate over all subjects and sessions
# =============================================================================

base_data_path = "../data"
subjects = range(1, 16)
sessions = range(1, 8)

results = []

for subj in subjects:
    for sess in sessions:
        eeg, labels = load_eeg(base_data_path, subj, sess)
        if eeg is None:
            continue

        eeg = preprocess_eeg(eeg)
        bp = compute_bandpower(eeg)
        p300_amp, p300_lat = compute_p300_features(eeg)

        results.append({
            "Subject": f"SBJ{subj:02d}",
            "Session": f"S{sess:02d}",
            "Mean_P300_Amplitude(µV)": np.mean(p300_amp),
            "Mean_P300_Latency(ms)": np.mean(p300_lat),
            "Mean_Delta_Power": np.mean(bp["delta"]),
            "Mean_Theta_Power": np.mean(bp["theta"]),
            "Mean_Alpha_Power": np.mean(bp["alpha"]),
            "Mean_Beta_Power": np.mean(bp["beta"]),
            "Signal_Variance": np.mean(np.var(eeg, axis=1))
        })

# =============================================================================
# 3️⃣ Add Normative Ranges (Ages 3–5 years)
# =============================================================================

df = pd.DataFrame(results)

# Norms based on pediatric EEG/ERP studies (3–5 years)
norms = {
    "Norm_P300_Amplitude_Lower(µV)": 6.0,
    "Norm_P300_Amplitude_Upper(µV)": 12.0,
    "Norm_P300_Latency_Lower(ms)": 350,
    "Norm_P300_Latency_Upper(ms)": 450,
    "Norm_Delta_Power_Lower": 0.25,
    "Norm_Delta_Power_Upper": 0.45,
    "Norm_Theta_Power_Lower": 0.25,
    "Norm_Theta_Power_Upper": 0.35,
    "Norm_Alpha_Power_Lower": 0.15,
    "Norm_Alpha_Power_Upper": 0.30,
    "Norm_Beta_Power_Lower": 0.05,
    "Norm_Beta_Power_Upper": 0.15,
    "Norm_Theta_Beta_Ratio_Lower": 2.0,
    "Norm_Theta_Beta_Ratio_Upper": 3.5,
}

# Compute actual theta/beta ratio
df["Theta_Beta_Ratio"] = df["Mean_Theta_Power"] / df["Mean_Beta_Power"]

# Add norm columns
for k, v in norms.items():
    df[k] = v

# =============================================================================
# 4️⃣ Save Results as CSV
# =============================================================================

df.to_csv("eeg_feature_markers_with_norms.csv", index=False)
print("✅ EEG marker features (with developmental norms) saved as 'eeg_feature_markers_with_norms.csv'")

print(df.head())


C:\Users\NILANJAN SAHA\AppData\Local\Temp\ipykernel_149828\1751366219.py:39: UserWarning: nperseg=500 is greater than signal length max(len(x), len(y)) = 350, using nperseg = 350
  freqs, psd = welch(eeg, fs=fs, axis=1, nperseg=fs*2)


✅ EEG marker features (with developmental norms) saved as 'eeg_feature_markers_with_norms.csv'
  Subject Session  Mean_P300_Amplitude(µV)  Mean_P300_Latency(ms)  \
0   SBJ01     S01                 0.707211                  444.5   
1   SBJ01     S02                 0.819100                  420.5   
2   SBJ01     S03                 0.748979                  403.5   
3   SBJ01     S04                 1.007567                  412.5   
4   SBJ01     S05                 1.006429                  392.0   

   Mean_Delta_Power  Mean_Theta_Power  Mean_Alpha_Power  Mean_Beta_Power  \
0         10.505694          4.918353          8.704689         0.711893   
1         11.660251          6.621754          9.320741         0.859473   
2         47.954302          7.326012          8.274683         1.396947   
3         10.102972          5.601228          7.187899         0.650331   
4         11.290894          5.641115          8.191019         0.842899   

   Signal_Variance  Theta_Beta_Ra